## Protein Generation and Structure Prediction with BioNemo

This example notebook shows how to generate new protein sequences and predict folded protein structures using ProtGPT2 and AlphaFold pre-trained models via the BioNeMo service API. These models were trained and deployed using NVIDIA's BioNeMo framework for Large Language Models. For more details, please visit NVIDIA BioNeMo Service at https://www.nvidia.com/en-us/gpu-cloud/bionemo/ 

This notebook will walk through protein generation and visualization in the following sections:

 - **BioNeMo Service Configuration**
   - Install dependencies and define the BioNeMo service endpoint and API key required for access
 - **Generating Protein Sequences**
   - Generate protein sequences with the ProtGPT2 model
 - **Predicting 3d Protein Structure**
   - Predict the 3D structure of the generated proteins using AlphaFold

### BioNeMo Service Configurations
To get started, please configure and provide your NGC access token by visiting https://ngc.nvidia.com/setup/api-key

In [1]:
API_KEY = "YOUR KEY HERE"
API_HOST = "https://api.bionemo.ngc.nvidia.com/v1"

Let's start by installing and importing library dependencies. We'll use _requests_ to interact with the BioNeMo service, and _py3Dmol_ for visualization.

In [ ]:
!pip install py3dmol

In [3]:
import time
import datetime
import py3Dmol

Next, we can configure the BioNeMo Service Client and validate our connection to the BioNeMo service host.  The BioNeMo Service Client is a Python interface to the BioNeMo Service API, exposing easy-to-use Python functions that allow users to direct access to state-of-the-art models with minimal configuration.  The BioNeMo Service Client is installed with a simple `pip install bionemo` and has been preinstalled in this container.

To get started, we'll first instantiate the BioNeMo Service Client with our API_KEY and API_HOST defined above, and then query the service for the list of available models.

In [ ]:
from bionemo.api import BionemoClient
api = BionemoClient(api_key=API_KEY, api_host=API_HOST)
models=api.list_models()
models

The BioNeMo Python client offers two methods for querying BioNeMo Service:
 - _synchronous_, in which the service API call will block until a result is returned.
 - _asynchronous_ or nonblocking, which immediately return a handle called a _correlation_id_ that can be used to query the results.

The BioNeMo Python client calls can be differentiated by their name.  Blocking _synchronous_ calls use the `_sync` suffix: `{model_id}_sync`, whereas non-blocking _asynchronous_ calls use the `_async` suffix: `{model_id}_async`.

We will show examples of both forms in the following sections.  Blocking calls will be used when querying the UniProt ID for a protein sequence and generating protein sequences with ProtGPT2.  Non-blocking calls are used when batching requests to generate protein structures with AlphaFold.


## Protein structure prediction via API request to BioNeMo Service

In the following, we will demonstrate an example of protein structure prediction utilizing sequences we generate from an AI model. In this case we'll be using ProtGPT2, a language model that generates protein sequences that are similar to their natural counterparts.  But before we generate our own sequences, let's take a look at the BioNeMo UniProt lookup service. This can be leveraged to retrieve the sequence of a protein-of-interest, using the UniProt ID as input. Here we will be looking at [Thioredoxin](https://www.uniprot.org/uniprotkb/P10599/entry). This small protein plays a vital role in regulating cellular redox (reduction–oxidation) homeostasis by reducing disulfide bonds in proteins. It is found in most living organisms, from bacteria to humans. It acts as a reducing agent by donating electrons to other proteins to help maintain their proper shape and function. It also helps to remove reactive oxygen species (ROS) from cells, which can damage DNA, proteins, and lipids if not removed.

Let's begin by retrieving the sequence for Thioredoxin, utilizing the BioNeMo UniProt lookup service.
<div class="alert alert-block alert-info">
    <b>Tip:</b>  We use the specific sequence for Thioredoxin in this example workflow.  The UniProt ID lookup feature built in to BioNeMo Service allows a user to work with virtually any protein sequence.  Feel free to use this workflow as a starting point for your own experimentation!
</div>

In [ ]:
uniprot_id="P10599"
original_sequence = api.get_uniprot(uniprot_id)
print(original_sequence)

Generating new protein sequences is a key component of protein engineering, which allows scientists to create proteins with specific functions and properties that may not exist in nature. These engineered proteins can be used in a variety of applications, including drug development, biocatalysis, and biomaterials, among others.

It's important to choose your parameters for the protein generation process carefully.

-   max_length: maximum number of generated tokens.
        
Note that common tokens in ProtGPT3 are k-mers of length 3, so 1 token = 3 amino acids. In other words, a max_length=400 could translate to an effective protein sequence of up to 1200 amino acids.

- top_k: Sets the number of highest probability vocabulary tokens to keep for top-k-filtering
- repetition_penalty: Sets the penalty for [repeating tokens](https://arxiv.org/pdf/1909.05858.pdf), where a value of 1 correspond to no penalty.
- num_return_sequences: The effective number of whole protein sequences to return
- percent_to_keep: Sets the percent of whole protein sequences to keep for each protein generation iteration cycle, based on their cumulative perplexity. 

As an illustrative example, suppose that 50 sequences of tokens are generated per iteration. From these generated sequences of tokens (which could contain protein sequence fragments or other invalid # sequences), whole protein sequences are reconstructed. Let's suppose that there are 43 valid, whole protein sequences from the generative iteration. Of these 43 whole protein sequences, only the top `percent_to_keep`, according to perplexity, are kept. For instance, with percent_to_keep=10, only ~4 of the 43 sequences (i.e., ~10% of 43) will be kept. After these 4 sequences are added to the return object, the generation process will resume until `num_return_sequences` is reached. Therefore - the lower the value of `percent_to_keep`, the longer the overall generation process will take.

Now let's define a sane set of parameters for our example problem.

Now we can submit our service request to BioNeMo. We'll use the blocking call `protgpt2_sync()` to generate results. With the chosen input parameters, we expect to obtain 5 new sequences.

In [ ]:
generated_sequences = api.protgpt2_sync(
    max_length=100,
    top_k=950,
    repetition_penalty=1.2,
    num_return_sequences=5,
    percent_to_keep=0.1
)
print(generated_sequences)

With our sequences in hand, we're ready to predict the corresponding protein structures using the BioNeMo folding services. We'll use AlphaFold to fold both the original protein and the first generated sequence using synchronous calls to the API.  These will be used to visualize the protein structures.

*Note: Alphafold can take up to 30 minutes to complete an API call.*

In [7]:
# First let's generate the protein structure for Human Thioredoxin as sourced from UniProt
# and the first generated sequence.

original_result=api.alphafold2_sync(original_sequence)
with open("data/BioNeMo_AlphaFold_original.pdb", "w") as f:
    f.write(original_result)
    
generated_result=api.alphafold2_sync(generated_sequences['generated_sequences'][0])
with open("data/BioNeMo_AlphaFold_generated.pdb", "w") as f:
    f.write(generated_result)

We can also take advantage of the asynchronous nature of the folding services.  Here we will batch and submit requests for the remaining generated sequences before waiting for any service response.  We'll then query the _correlation_ids_ to write the results to files once complete.  This can be useful when querying large batches of input sequences.

_Note: this can take some time.  Feel free to skip the following two cells and carry on with visualization below._

In [ ]:
# First we'll submit asynchronous requests for the remaining generated sequences.
correlation_ids=[]
for sequence in generated_sequences['generated_sequences'][1:len(generated_sequences['generated_sequences'])]:
    request_id=api.alphafold2_async(sequence)
    correlation_ids.append(request_id)
print(correlation_ids)

In [27]:
# Now we can query the correlation_ids and write the resulting PDB to a file when ready.
while len(correlation_ids):
    time.sleep(10)
    for request_id in correlation_ids:
        if api.fetch_task_status(request_id) == "DONE":
            alphafold_result=api.fetch_result(request_id)
            with open(f'data/{request_id}.pdb', "w") as f:
                f.write(alphafold_result)
            correlation_ids.remove(request_id)

### Understanding the asynchronous response

The BioNeMo Server will respond with a `correlation_id`, indicating a unique identifier for the request. Once the request is submitted, it is queued for processing. As soon as a processing slot is available, the structure prediction process is started. You can keep an eye on the submission request by querying the Server with the `correlation_id`.

In the code above, we waited for the status to be completed in order to download the structure prediction and save it into a pdb file that we then can visualize.

More information about the API can be found here: https://developer.nvidia.com/docs/bionemo-service/working-with-the-api.html

## Visualizing the predicted structures and prediction confidence

Finally, we visualize the predictions in 3D using [*py3Dmol*](https://pypi.org/project/py3Dmol/).

We take advantage of the predicted IDDT, i.e., a proxy to model confidence, and visualize it as a color similar to AlpaFold2.  In this color scheme, we bin the confidence intervals as:
 - <span style="color:blue">&#11035;</span> (dark blue, very high) - 90-100%</span>
 - <span style="color:lightblue">&#11035;</span> (light blue, confident) - 70-90%
 - <span style="color:yellow">&#11035;</span> (yellow, low) - 50-70%
 - <span style="color:orange">&#11035;</span> (orange, very low) - <50%

Regions in dark blue, with very high confidence, are expected to be modeled with high accuracy and can be used in applications that require high accuracy such as identifying binding sites.  Light blue regions with 70-90% confidence are expected to model general structure well.  Regions with low 50-70% confidence should be treated with caution, and below 50% confidence should not be used.

In the visualizations below, we show the two cases from the examples above:
 - The predicted structure of the original Thioredoxin sequence from the UniProt sequence look-up service, and
 - The predicted structure of the novel protein sequence generated with ProtGPT
 
Note that these are independent structures.  The following visualizations are not intended for comparison, but rather to showcase the ability to predict 3D structure using AlphaFold.

In [8]:
# Define color palette for IDDT
color_palette = {
    range(90,100): 'blue',
    range(70,90): 'lightblue',
    range(50,70): 'yellow',
    range(0,50): 'orange',
}

def get_color(IDDT):
    for key in color_palette:
        if IDDT in key:
            return color_palette[key]

First the original Thioredoxin structure:

In [ ]:
filename="data/BioNeMo_AlphaFold_original.pdb"
# Loading the predicted structure saved in PDB file
with open(filename) as ifile:
    system = "".join([x for x in ifile])

#configuring the structure display
view = py3Dmol.view(width=800, height=800)
view.addModelsAsFrames(system)

# Iterate over residues and color based on IDDT value
for i, line in enumerate(system.split("\n")):
    split = line.split()
    if len(split) == 0 or split[0] != "ATOM":
        continue

    color = get_color(int(float(split[10])))
    
    view.setStyle({'model': -1, 'serial': i+1}, {"cartoon": {'color': color}})

view.zoomTo()
view.show()

Next the predicted structure of the generated sequence.

In [ ]:
filename="data/BioNeMo_AlphaFold_generated.pdb"
# Loading the predicted structure saved in PDB file
with open(filename) as ifile:
    system = "".join([x for x in ifile])

#configuring the structure display
view = py3Dmol.view(width=800, height=800)
view.addModelsAsFrames(system)

# Iterate over residues and color based on IDDT value
for i, line in enumerate(system.split("\n")):
    split = line.split()
    if len(split) == 0 or split[0] != "ATOM":
        continue

    color = get_color(int(float(split[10])))
    
    view.setStyle({'model': -1, 'serial': i+1}, {"cartoon": {'color': color}})

view.zoomTo()
view.show()

## Conclusion
In this notebook, we've walked through an example workflow generating protein sequences and visualizing their 3D structure, covering:
 - How to configure the BioNeMo Service API
 - Generating protein sequences with the ProtGPT2 model
 - Predicting 3D protein structure of the generated sequences using AlphaFold
 
While this notebook demonstrates some of the rich capabilities of the BioNeMo service, this could just be the beginning of a production end-to-end drug discovery pipeline.